In [1]:
import torch.nn as nn
import torch
import matplotlib.pyplot as plt 
import pandas as pd
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from sklearn.datasets import load_iris
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import tqdm

from sklearn.cluster import KMeans

from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

/home/menon/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def plot_confusion_matrix(cm,
                          target_names=None,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=False):
    """
    given a sklearn confusion matrix (cm), make a nice plot

    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix

    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']

    title:        the text to display at the top of the matrix

    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues

    normalize:    If False, plot the raw numbers
                  If True, plot the proportions

    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph

    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    """
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / np.sum(cm).astype('float')
    misclass = 1 - accuracy

    if target_names is None:
        target_names = [i for i in range(cm.shape[0])]

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

In [3]:
def transf_dados(name):
    d = {}  # dictionary that will hold them 

    for i in range(1,24):  # loop over files
        if i < 10: count = '0'+str(i)
        else: count= str(i)

        file_name = "P"+count+"_"+name
        # read csv into a dataframe and add it to dict with file_name as it key
        d[file_name] = pd.read_csv('dataset/'+file_name+'.csv')
        
    x_values =np.zeros(0)
    y_values =np.zeros(0)
    keys=[]
    for key in d:
        _len = len(d[key]['x'])
        time_series = np.zeros((3,_len))
        keys.append(key)

    return d, keys


In [4]:
d1,keys1 = transf_dados('READ')
d2,keys2 =transf_dados('SEARCH')

d3,keys3 =transf_dados('BROWSE')
d4,keys4 =transf_dados('DEBUG')
d5,keys5 =transf_dados('INTERPRET')
d6,keys6 =transf_dados('PLAY')
d7,keys7 =transf_dados('WATCH')
d8,keys8 =transf_dados('WRITE')


In [5]:
D={}
dd1 ={} 
dd2 ={}
dd3 ={} 
dd4 ={}
dd5 ={} 
dd6 ={}
dd7 ={} 
dd8 ={}

dd1['p01'] = d1['P01_READ']
dd2['p01'] = d2['P01_SEARCH']

dd3['p01'] = d3['P01_BROWSE']
dd4['p01'] = d4['P01_DEBUG']

dd5['p01'] = d5['P01_INTERPRET']
dd6['p01'] = d6['P01_PLAY']

dd7['p01'] = d7['P01_WATCH']
dd8['p01'] = d8['P01_WRITE']

D['READ'] = dd1
D['SEARCH'] = dd2

D['BROWSE'] = dd3
D['DEBUG'] = dd4

D['INTERPRET'] = dd5
D['PLAY'] = dd6

D['WATCH'] = dd7
D['WRITE'] = dd8

In [6]:
window =4
lst_labels =[]
lst_values =[]
columns = {}


for index in range(window):
    label_x = "x"+str(index)
    label_y = "y"+str(index)
    lst_labels.insert(len(lst_labels), label_x)
    lst_labels.insert(len(lst_labels), label_y)
    columns['label_x'] = 'numeric'
    columns['label_y'] = 'numeric'
lst_labels.insert(len(lst_labels),'ROTULO')
columns['ROTULO'] ='categorical'

for atividade in D:
    for pessoa in D[atividade]:
        for j in range(1,5500,window):
            lst_values_aux =[] 
            for i in range(window):
                            
                lst_values_aux.insert(len(lst_values_aux), D[atividade][pessoa]['x'][j+i])
                lst_values_aux.insert(len(lst_values_aux), D[atividade][pessoa]['y'][j+i])

            lst_values_aux.insert(len(lst_values_aux),atividade)
            lst_values.insert(len(lst_values),lst_values_aux)
            
df2 = pd.DataFrame([lst_values[0]], columns = lst_labels)           
for j in range(1,len(lst_values)):
    newdf = pd.DataFrame([lst_values[j]], columns = lst_labels)               
    df2 = pd.concat([df2, newdf])

In [7]:
label_encoder = LabelEncoder()
label =label_encoder.fit_transform(df2['ROTULO'])
df2['class'] = label

scaler_1 = MinMaxScaler()


df2[lst_labels[:-1]] = scaler_1.fit_transform(df2[lst_labels[:-1]])
df2 = df2.reset_index(drop=True)
df2.groupby(['class']).head(1)

,x0,y0,x1,y1,x2,y2,x3,y3,ROTULO,class
0,0.738480,0.523135,0.636020,0.638465,0.601935,0.482245,0.626700,0.525903,READ,4
1375,0.651893,0.517610,0.531971,0.618840,0.464286,0.403207,0.410795,0.502386,SEARCH,5
2750,0.742882,0.510359,0.638668,0.625818,0.607515,0.469645,0.636588,0.517382,BROWSE,0
4125,0.549163,0.464088,0.390087,0.561710,0.359747,0.383734,0.323445,0.469666,DEBUG,1
5500,0.601702,0.535566,0.367764,0.663323,0.343378,0.528637,0.319324,0.478868,INTERPRET,2
6875,0.499853,0.359116,0.320091,0.419102,0.321057,0.206186,0.313968,0.356510,PLAY,3
8250,0.587907,0.714434,0.496406,0.872656,0.447917,0.816724,0.494850,0.723586,WATCH,6
9625,0.639859,0.501036,0.444192,0.593982,0.419271,0.435281,0.416976,0.494888,WRITE,7


In [8]:
X = pd.DataFrame.to_numpy(df2[lst_labels[:-1]])
Y = pd.Series.to_numpy(df2['class'])

In [9]:
#x, x_val, y, y_val = train_test_split(X, Y, test_size=0.33, random_state=42)
x, x_test, y, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
x_test, x_val, y_test, y_val = train_test_split(x_test,y_test,test_size=0.2,random_state=42)

In [10]:
x.shape, y.shape, x_val.shape, y_val.shape



((8800, 8), (8800,), (440, 8), (440,))

In [11]:
x_train = x.reshape(-1, x.shape[1]).astype('float32')
y_train = y

x_val = x_val.reshape(-1, x_val.shape[1]).astype('float32')
y_val = y_val

In [12]:
class Data(Dataset):
    def __init__(self):
        self.x=torch.from_numpy(x_train).float()
        self.y=torch.from_numpy(y_train).long()
        self.len=self.x.shape[0]
    def __getitem__(self,index):      
        return self.x[index], self.y[index]
    def __len__(self):
        return self.len
    
class Data_ver(Dataset):
    def __init__(self):
        self.x_val=torch.from_numpy(x_val).float()
        self.y_val=torch.from_numpy(y_val).long()
        self.x_test=torch.from_numpy(x_test)
        self.y_test=torch.from_numpy(y_test).long()
        #self.len=self.x.shape[0]
    def __getitem__(self,index):      
        return self.x[index], self.y[index]
    def __len__(self):
        return self.len

In [13]:
data_set    = Data()
ver_dataset  = Data_ver()

In [14]:
trainloader=DataLoader(dataset=data_set,batch_size=1)

In [15]:
data_set.x.shape, data_set.y.shape

(torch.Size([8800, 8]), torch.Size([8800]))

In [16]:
class Net(nn.Module):
    def __init__(self,D_in,H,D_out):
        super(Net,self).__init__()
        self.linear1=nn.Linear(D_in,H)
        self.linear2=nn.Linear(H,D_out)

        
    def forward(self,x):
        x=torch.sigmoid(self.linear1(x))  
        x=self.linear2(x)
        return x

In [17]:
input_dim=window*2   # how many Variables are in the dataset
hidden_dim = 14 # hidden layers
output_dim=8   # number of classes

In [18]:
# Instantiate model
model=Net(input_dim,hidden_dim,output_dim)
criterion=nn.CrossEntropyLoss()
learning_rate=0.1
optimizer=torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
n_epochs=100
#loss_list=[]
batch_size=12
loss_list     = np.zeros((n_epochs,))
accuracy_list = np.zeros((n_epochs,))
early_stop = False
trigger_times =0
patience =2
for epoch in tqdm.trange(n_epochs):
    for x, y in trainloader:
      

        #clear gradient 
        optimizer.zero_grad()
        #make a prediction 
        z=model(x)
        # calculate loss, da Cross Entropy benutzt wird muss ich in den loss Klassen vorhersagen, 
        # also Wahrscheinlichkeit pro Klasse. Das mach torch.max(y,1)[1])
        loss=criterion(z,y)
        # calculate gradients of parameters 
        loss.backward()
        # update parameters 
        optimizer.step()
        
        
        loss_list[epoch] = loss.data
        '''if epoch>10:
            if loss_list[epoch]> round(loss_list[epoch-1],1):
                trigger_times += 1
                if trigger_times >= patience:
                        print('Early stopping!\nStart to test process.')
                        early_stop = True
                        break
            else:
                   # print('trigger times: 0')
                    trigger_times = 0''' 
        
    with torch.no_grad():
        y_pred = model(ver_dataset.x_val)
        correct = (torch.argmax(y_pred, dim=1) == ver_dataset.y_val).type(torch.FloatTensor)
        accuracy_list[epoch] = correct.mean()
    if early_stop == True:
        final_epoch = epoch
        break
        #print('epoch {}, loss {}'.format(epoch, loss.item()))

  2%|▊                                          | 2/100 [00:07<06:24,  3.92s/it]

In [ ]:
y_pred=model(ver_dataset.x_test.type(torch.FloatTensor))

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, figsize=(12, 6), sharex=True)

ax1.plot(accuracy_list[:epoch])
ax1.set_ylabel("validation accuracy")
ax2.plot(loss_list[:epoch])
ax2.set_ylabel("validation loss")
ax2.set_xlabel("epochs");


In [ ]:
pred_arr = y_pred.detach().numpy()
original_arr = ver_dataset.y_test.detach().numpy()

In [ ]:
new_pred_arr = np.zeros(len(pred_arr))
for i in range(len(pred_arr)):
    new_pred_arr[i] = np.argmax(pred_arr[i])

In [ ]:
print(classification_report(original_arr, new_pred_arr))

In [ ]:
cm = confusion_matrix(original_arr, new_pred_arr)

plot_confusion_matrix(cm)

In [ ]:
f = open("acc_lss_1.dat", 'w+')
for i in range(len(accuracy_list[:epoch])):
    f.write('%f \t %f \t %f \n'%(i,accuracy_list[i],loss_list[i]))
f.close()

np.savetxt("matrix.dat", np.transpose(cm),fmt="%3.0f",  delimiter='\t ', newline='\n')